# Download and process sentinel 1 data

## John Brandt
## Nov 17, 2020

## Package imports, API import, source scripts

In [1]:
import datetime
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import scipy.sparse as sparse
import seaborn as sns
import yaml

from collections import Counter
from osgeo import ogr, osr
from random import shuffle
from scipy.sparse.linalg import splu
from sentinelhub import WmsRequest, WcsRequest, MimeType
from sentinelhub import CRS, BBox, constants, DataSource, CustomUrlParam
from skimage.transform import resize

import reverse_geocoder as rg
import pycountry
import pycountry_convert as pc
import hickle as hkl
from shapely.geometry import Point, Polygon

with open("../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key'] 
        
%matplotlib inline
%run ../src/downloading/utils.py

In [2]:
time = ('2019-12-15', '2021-01-15')
YEAR = 2020
IMSIZE = 24

# Bounding boxes

In [3]:
def identify_s1_layer(coords):
    coords = (coords[1], coords[0])
    results = rg.search(coords)
    admin1 = (results[-1]['admin1'])
    admin2 = results[-1]['admin2']
    country = results[-1]['cc']
    continent_name = pc.country_alpha2_to_continent_code(country)
    print(admin1, admin2, country, continent_name)
    if continent_name in ['AF', 'OC', 'EU']:
        layer = "SENT"
    if continent_name in ['SA']:
        if coords[0] > -7.11:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['AS']:
        if coords[0] > 23.3:
            layer = "SENT"
        else:
            layer = "SENT_DESC"
    if continent_name in ['NA']:
        layer = "SENT_DESC"
    return layer


def calc_bbox(plot_id, df):
    """ Calculates the corners of a bounding box from an input
        pandas dataframe as output by Collect Earth Online

        Parameters:
         plot_id (int): plot_id of associated plot
         df (pandas.DataFrame): dataframe of associated CEO survey
    
        Returns:
         bounding_box (list): [(min(x), min(y)),
                              (max(x), max_y))]
    """
    subs = df[df['PLOT_ID'] == plot_id]
    return [(min(subs['LON']), min(subs['LAT'])),
            (max(subs['LON']), max(subs['LAT']))]


def bounding_box(points, expansion = 160):
    """ Calculates the corners of a bounding box with an
        input expansion in meters from a given bounding_box
        
        Subcalls:
         calculate_epsg, convertCoords

        Parameters:
         points (list): output of calc_bbox
         expansion (float): number of meters to expand or shrink the
                            points edges to be
    
        Returns:
         bl (tuple): x, y of bottom left corner with edges of expansion meters
         tr (tuple): x, y of top right corner with edges of expansion meters
    """
    bl = list(points[0])
    tr = list(points[1])
    inproj = Proj('epsg:4326')
    outproj_code = calculate_epsg(bl)
    outproj = Proj('epsg:' + str(outproj_code))
    
    bl_utm =  transform(inproj, outproj, bl[1], bl[0])
    tr_utm =  transform(inproj, outproj, tr[1], tr[0])

    distance1 = tr_utm[0] - bl_utm[0]
    distance2 = tr_utm[1] - bl_utm[1]
    expansion1 = (expansion - distance1)/2
    expansion2 = (expansion - distance2)/2
    
    bl_utm = [bl_utm[0] - expansion1, bl_utm[1] - expansion2]
    tr_utm = [tr_utm[0] + expansion1, tr_utm[1] + expansion2]
    
    zone = str(outproj_code)[3:]
    zone = zone[1:] if zone[0] == "0" else zone
    direction = 'N' if tr[1] >= 0 else 'S'
    utm_epsg = "UTM_" + zone + direction
    return (bl_utm, tr_utm), CRS[utm_epsg]

# Data download

In [4]:
def extract_dates(date_dict: dict, year: int) -> List:
    """ Transforms a SentinelHub date dictionary to a
         list of integer calendar dates
    """
    dates = []
    days_per_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
    starting_days = np.cumsum(days_per_month)
    for date in date_dict:
        if date.year == year - 1:
            dates.append(-365 + starting_days[(date.month-1)] + date.day)
        if date.year == year:
            dates.append(starting_days[(date.month-1)] + date.day)
        if date.year == year + 1:
            dates.append(365 + starting_days[(date.month-1)]+date.day)
    return dates

def identify_dates_to_download(dates):
    dates = np.array(dates)
    dates_to_download = []
    for i in starting_days:
        s1_month = dates[dates > i]
        s1_month = s1_month[s1_month < (i + 30)]
        if len(s1_month) > 0:
            dates_to_download.append(s1_month[0])
    return dates_to_download

def to_float32(array: np.array) -> np.array:
    """Converts an int_x array to float32"""
    print(f'The original max value is {np.max(array)}')
    if not isinstance(array.flat[0], np.floating):
        assert np.max(array) > 1
        array = np.float32(array) / 65535.
    assert np.max(array) <= 1
    return array

def download_sentinel_1(bbox, epsg, time = time, 
                        layer = "SENT", year = 2020, image_format = MimeType.TIFF_d16, data = DataSource.SENTINEL1_IW_ASC):
    """ Downloads all 10 and 20 meter L2A bands from sentinel-hub
        for input bbox and epsg, within time range
        
        Parameters:
         bbox (list): output of calc_bbox
         epsg (float): UTM EPSG associated with bbox 
         time (tuple): YY-MM-DD - YY-MM-DD bounds for downloading 
    
        Returns:
         s1 (arr): (Time, X, Y, 2) array of sentinel 1 data
         image_dates (list): number of days since time[0] for each
                              image in s1.shape[0]
    """
    try:
        print(f"The data is {data}")
        box = BBox(bbox, crs = epsg)
        image_request = WcsRequest(
                layer=layer,
                bbox=box,
                time=time,
                image_format = image_format,
                data_source= data,
                maxcc=1.0,
                resx='10m', resy='10m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                    constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=72),
            )
        
        
        s1_dates_dict = [x for x in image_request.get_dates()]
        s1_dates = extract_dates(s1_dates_dict, year)
        dates_to_download = identify_dates_to_download(s1_dates)
        
        steps_to_download = [i for i, val in enumerate(s1_dates) if val in dates_to_download]
        print(f"The following steps will be downloaded: {steps_to_download}, for {dates_to_download}")
        
        
        data_filter = steps_to_download
        if len(image_request.download_list) == 0:
            return np.empty((0,)), np.empty((0,))
        s1 = image_request.get_data(data_filter = data_filter)
        s1 = np.stack(s1)
        s1 = to_float32(s1)
        
        assert np.max(s1) <= 1.
        assert s1.shape[1] == 24.
        assert s1.shape[2] == 24.
        
        print(f"Sentinel 1 used {(2/3)*s1.shape[0] * (s1.shape[1]*s1.shape[2])/(512*512)} PU for"
              f" {s1.shape[0]} out of {len(image_request.download_list)} images")
        
        
        image_dates = []
        for date in image_request.get_dates():
            if date.year == year - 1:
                image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
            if date.year == year:
                image_dates.append(starting_days[(date.month-1)] + date.day)
            if date.year == year + 1:
                image_dates.append(365 + starting_days[(date.month-1)]+date.day)
        image_dates = [val for idx, val in enumerate(image_dates) if idx in data_filter]
        image_dates = np.array(image_dates)
        
        s1c = np.copy(s1)
        s1c[np.where(s1c < 1.)] = 0
        n_pix_oob = np.sum(s1c, axis = (1, 2, 3))
        to_remove = np.argwhere(n_pix_oob > (16*16)/5)
        if len(to_remove) > 0:
            print(f'A total of {len(to_remove)} steps of {s1.shape[0]} were removed.')
            s1 = np.delete(s1, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)

        return s1, image_dates

    except Exception as e:
        logging.fatal(e, exc_info=True)

# Download function

In [5]:
starting_days = np.cumsum([0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30])
print(starting_days)

[  0  31  59  90 120 151 181 212 243 273 304 334]


In [6]:
def download_plots(data_location, output_folder, image_format = MimeType.TIFF_d16):
    """ Downloads sentinel-1 data for the plot IDs associated
        with an input CSV from a collect earth online survey
        
        Parameters:
         data_location (os.path)
         output_folder (os.path)
        
        Subcalls:
         calc_bbox, bounding_box
         download_sentinel_1,
         calculate_and_save_best_images
         
        Creates:
         output_folder/{plot_id}.npy
    
        Returns:
         None
    """
    df = pd.read_csv(data_location, encoding = "ISO-8859-1")
    df.columns = [x.upper() for x in df.columns]
    for column in ['IMAGERY_TITLE', 'STACKINGPROFILEDG', 'PL_PLOTID', 'IMAGERYYEARDG',
                  'IMAGERYMONTHPLANET', 'IMAGERYYEARPLANET', 'IMAGERYDATESECUREWATCH',
                  'IMAGERYENDDATESECUREWATCH', 'IMAGERYFEATUREPROFILESECUREWATCH',
                  'IMAGERYSTARTDATESECUREWATCH',
                  'IMAGERY_ATTRIBUTIONS',
                  'SAMPLE_GEOM']:
        if column in df.columns:
            df = df.drop(column, axis = 1)
    df = df.dropna(axis = 0)
    plot_ids = sorted(df['PLOT_ID'].unique())
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]

    to_download = [x for x in plot_ids if x not in existing]
    print(f"Starting download of {len(to_download)}"
          f" plots from {data_location} to {output_folder}")
    errors = []
    for i, val in enumerate(to_download):
        print(f"Downloading {i+1}/{len(to_download)}, {val}")
        location_wgs = calc_bbox(val, df = df)
        location, epsg = bounding_box(location_wgs, expansion = IMSIZE*10)
        try:
            # Identify cloud steps, download DEM, and download L2A series
            s1_layer = identify_s1_layer(location_wgs[0])
            data_source = DataSource.SENTINEL1_IW_DES if s1_layer == "SENT_DESC" else DataSource.SENTINEL1_IW_ASC
            s1, s1_dates = download_sentinel_1(location, 
                                               layer = s1_layer, 
                                               epsg = epsg,
                                               data = data_source)
            if s1.shape[0] < 5:
                s1_layer = "SENT_DESC" if s1_layer == "SENT" else "SENT"
                data_source = DataSource.SENTINEL1_IW_DES if s1_layer == "SENT_DESC" else DataSource.SENTINEL1_IW_ASC
                print(f'Switching to {s1_layer}')
                s1, s1_dates = download_sentinel_1(location, 
                                                   layer = s1_layer,
                                                   epsg = epsg,
                                                   data = data_source)
            
            s1_a = np.copy(s1)
            s1, max_distance = calculate_and_save_best_images(s1, s1_dates)

            s1_b = np.copy(s1)
            # Retain only iamgery every month
            monthly = np.empty((12, 24, 24, 2))
            index = 0
            for start, end in zip(range(0, 72 + 6, 72 // 12), #0, 72, 6
                                  range(72 // 12, 72 + 6, 72 // 12)): # 6, 72, 6
                monthly[index] = np.median(s1[start:end], axis = 0)
                index += 1

            s1 = monthly
            s1_c = np.copy(s1)
            
            assert s1.shape[1] == 24
            assert s1.shape[2] == 24
            print(s1.shape)
            if max_distance < 200:
                np.save(output_folder + str(val), s1)
                print('\n')
            else:
                print(f"Skipping {val} because max distance is {max_distance}")
            
        except Exception as e:
            print(e)
            logging.fatal(e, exc_info=True)
            errors.append(i)

In [8]:
for i in (os.listdir("../data/train-csv/")):
    if "cafrica" in i:
        #if any(x in i for x in ["africa-west", "cameroon", "koure", "niger"]):
        download_plots("../data/train-csv/" + i, "../data/train-s1-radiometric/", image_format = MimeType.TIFF_d16)

Starting download of 152 plots from ../data/train-csv/ceo-cafrica-train-sample-data-2021-02-16.csv to ../data/train-s1-radiometric/
Moxico  AO AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 17, 19, 22, 24, 27, 29], for [2, 38, 61, 97, 121, 157, 193, 217, 253, 277, 313, 337]
The original max value is 38465
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Northern  ZM AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 16, 19, 21, 24, 26, 29], for [6, 42, 65, 101, 125, 161, 185, 221, 245, 281, 305, 341]
The original max value is 52636
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Bie  AO AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [2, 4, 6, 9, 11, 13, 16, 18, 21, 22, 25, 27], for [12, 36, 60, 95, 131, 155, 191, 215, 251, 275, 311, 335]
The o

Morogoro  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 16, 19, 22, 24, 27, 29], for [4, 40, 63, 99, 123, 159, 183, 219, 255, 279, 315, 339]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Katavi  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 16, 19, 21, 24, 26, 29], for [7, 43, 66, 102, 126, 162, 186, 222, 246, 282, 306, 342]
The original max value is 65406
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Kasai-Occidental  CD AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [], for []
Switching to SENT_DESC
The data is DataSource.SENTINEL1_IW_DES
The following steps will be downloaded: [2, 4, 6, 9, 12, 14, 17, 19, 22, 24, 27, 29], for [12, 36, 60, 95, 131, 155, 191, 215, 251, 275, 311, 335]
The origin

Shinyanga  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 17, 19, 22, 24, 27, 29], for [2, 38, 61, 97, 121, 157, 193, 217, 253, 277, 313, 337]
The original max value is 50038
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Shinyanga  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 17, 19, 22, 24, 27, 29], for [2, 38, 61, 97, 121, 157, 193, 217, 253, 277, 313, 337]
The original max value is 57913
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Manyara  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 16, 19, 22, 24, 27, 29], for [4, 40, 63, 99, 123, 159, 183, 219, 255, 279, 315, 339]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)




Kagera  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [2, 8, 12, 18, 23, 28, 33, 38, 43, 48, 53, 58], for [1, 37, 60, 96, 126, 156, 186, 216, 246, 276, 306, 336]
The original max value is 38715
Sentinel 1 used 0.017578125 PU for 12 out of 66 images
Maximum time distance: 36
(12, 24, 24, 2)


Bandundu  CD AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [], for []
Switching to SENT_DESC
The data is DataSource.SENTINEL1_IW_DES
The following steps will be downloaded: [3, 8, 13, 18, 23, 28, 33, 38, 44, 49, 54, 59], for [3, 34, 62, 93, 122, 153, 182, 213, 249, 278, 309, 338]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 66 images
Maximum time distance: 36
(12, 24, 24, 2)


Bas-Congo  CD AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [], for []
Switching to SENT_DESC
The data is DataSource.SENTINEL1_IW_DES
The following steps will be downloaded: [2, 4, 

The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 65 images
A total of 1 steps of 12 were removed.
Maximum time distance: 60
(12, 24, 24, 2)


Plateaux  CG AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [], for []
Switching to SENT_DESC
The data is DataSource.SENTINEL1_IW_DES
The following steps will be downloaded: [2, 4, 7, 9, 12, 15, 17, 20, 22, 25, 27, 30], for [8, 32, 67, 91, 127, 163, 187, 223, 247, 283, 307, 343]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 34 images
Maximum time distance: 36
(12, 24, 24, 2)


Bomet  KE AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 16, 19, 22, 24, 27, 29], for [4, 40, 63, 99, 123, 159, 183, 219, 255, 279, 315, 339]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Narok  KE AF
The data is DataSource.SENTINEL1_IW_ASC


Cuvette  CG AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [], for []
Switching to SENT_DESC
The data is DataSource.SENTINEL1_IW_DES
The following steps will be downloaded: [2, 4, 7, 9, 12, 15, 17, 20, 22, 25, 27, 30], for [8, 32, 67, 91, 127, 163, 187, 223, 247, 283, 307, 343]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 34 images
Maximum time distance: 36
(12, 24, 24, 2)


Eastern Province  CD AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [], for []
Switching to SENT_DESC
The data is DataSource.SENTINEL1_IW_DES
The following steps will be downloaded: [1, 4, 6, 9, 11, 14, 16, 19, 21, 24, 26, 28], for [7, 43, 66, 102, 126, 162, 186, 222, 246, 282, 318, 342]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 32 images
Maximum time distance: 36
(12, 24, 24, 2)


Western Region Kamwenge District UG AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps

Tete  MZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [1, 4, 6, 9, 12, 14, 17, 19, 22, 24, 27, 29], for [1, 37, 60, 96, 132, 156, 192, 216, 252, 276, 312, 336]
The original max value is 45382
Sentinel 1 used 0.017578125 PU for 12 out of 33 images
Maximum time distance: 36
(12, 24, 24, 2)


Western  ZM AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [3, 8, 13, 18, 23, 28, 34, 39, 44, 49, 54, 59], for [2, 33, 61, 92, 121, 152, 188, 217, 248, 277, 308, 337]
The original max value is 65535
Sentinel 1 used 0.017578125 PU for 12 out of 67 images
Maximum time distance: 36
(12, 24, 24, 2)


Southern Region Chiradzulu District MW AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [2, 4, 7, 9, 12, 15, 17, 20, 22, 25, 27, 30], for [8, 32, 67, 91, 127, 163, 187, 223, 247, 283, 307, 343]
The original max value is 59309
Sentinel 1 used 0.017578125 PU for 12 out of 34 images
Maximum time distance:

CRITICAL:root:zero-size array to reduction operation minimum which has no identity
Traceback (most recent call last):
  File "<ipython-input-6-64a68c86c049>", line 60, in download_plots
    s1, max_distance = calculate_and_save_best_images(s1, s1_dates)
  File "/Users/john.brandt/Documents/GitHub/restoration-mapper/src/downloading/utils.py", line 216, in calculate_and_save_best_images
    closest = np.min(abs(distances))
  File "<__array_function__ internals>", line 6, in amin
  File "/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/numpy/core/fromnumeric.py", line 2831, in amin
    keepdims=keepdims, initial=initial, where=where)
  File "/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/numpy/core/fromnumeric.py", line 87, in _wrapreduction
    return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
ValueError: zero-size array to reduction operation minimum which has no identity


The following steps will be downloaded: [], for []
zero-size array to reduction operation minimum which has no identity
Mtwara  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [3, 8, 13, 18, 23, 28, 33, 38, 43, 48, 54, 59], for [5, 35, 64, 94, 124, 154, 184, 214, 244, 274, 310, 340]
The original max value is 31420
Sentinel 1 used 0.017578125 PU for 12 out of 66 images
Maximum time distance: 36
(12, 24, 24, 2)


Mtwara  TZ AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [3, 8, 13, 18, 23, 28, 33, 38, 44, 49, 54, 59], for [4, 34, 63, 93, 123, 153, 183, 213, 249, 279, 309, 339]
The original max value is 49106
Sentinel 1 used 0.017578125 PU for 12 out of 66 images
Maximum time distance: 36
(12, 24, 24, 2)


Katanga  CD AF
The data is DataSource.SENTINEL1_IW_ASC
The following steps will be downloaded: [], for []
Switching to SENT_DESC
The data is DataSource.SENTINEL1_IW_DES
The following steps will be downloaded: [2, 4